In [1]:
import pprint
from boardgamegeek import BGGClient
from boardgamegeek import CacheBackendSqlite
import pandas as pd
from pytictoc import TicToc


In [2]:
# Create an instance of the API handler:
pathToCache = "/Users/taylorsmith/Python/BGG/cache6.db"
bgg = BGGClient(timeout=15, retries=10, retry_delay=5, disable_ssl=False, requests_per_minute=30,cache=CacheBackendSqlite(path=pathToCache, ttl=3600))

# Test on my uname
u = bgg.user('Brunty_')

In [3]:
# Test for a game with a lot of ratings to make sure that the caching works, time to check feasibility
# Get all the ratings and information about this specific game
t = TicToc() #create instance of class
thisGame = 'Gloomhaven' # Example of a game with many ratings
thisGame = "Yukon Salon" # Example of a game with few ratings

t.tic() #Start timer
try:
    g = bgg.game(game_id=bgg.get_game_id(thisGame),rating_comments=True,comments=False)
    t.toc() #Time elapsed since t.tic()
    print('All of {} downloaded correctly'.format(thisGame))
except:
    t.toc() #Time elapsed since t.tic()
    print('There was an error in the information of {}'.format(thisGame))

'{} of the ratings for {} are recorded'.format(len(g.comments),thisGame)

Elapsed time is 3.799470 seconds.
All of Yukon Salon downloaded correctly


'37 of the ratings for Yukon Salon are recorded'

In [4]:
# Get the list of hot games, show it
hot = bgg.hot_items('boardgame')

statusList = []
gameList = []
print('The hot list of games from BoardGameGeek.com is:\n')
cnt = 1
for item in hot.items:
    print(str(cnt)+' '+item.name)
    cnt +=1
    statusList.append(False) # Initialize all games to not done
    gameList.append(item.id)

# Make the list of the hot games to add to the list of ones to process
progressList = dict(zip(gameList,statusList))

The hot list of games from BoardGameGeek.com is:

1 Stardew Valley: The Board Game
2 Tiny Epic Dungeons
3 So, You've Been Eaten.
4 Sleeping Gods
5 Everdell
6 Red Rising
7 City of the Big Shoulders
8 Root
9 Dune: Imperium
10 ALIEN: Fate of the Nostromo
11 Lost Ruins of Arnak
12 Great Western Trail
13 Terraforming Mars
14 Gloomhaven: Jaws of the Lion
15 Gloomhaven
16 John Company: Second Edition
17 Wingspan
18 Marvel Champions: The Card Game
19 Spirit Island
20 Everdell: Newleaf
21 Brass: Birmingham
22 Arkham Horror: The Card Game
23 Bloodborne: The Board Game
24 Robinson Crusoe: Adventures on the Cursed Island
25 Scythe
26 Tainted Grail: The Fall of Avalon
27 Root: The Marauder Expansion
28 Everdell: Mistwood
29 Terraforming Mars: Ares Expedition
30 Eclipse: Second Dawn for the Galaxy
31 Barrage
32 Canvas
33 John Company
34 Gaia Project
35 Res Arcana
36 The Crew: The Quest for Planet Nine
37 Nemesis
38 Meeples and Monsters
39 Faiyum
40 Centurion
41 Maracaibo
42 Dwellings of Eldervale
43

In [5]:
# Initialize the empty ratingList
ratingList = []


In [6]:
# For a given list of games, get the data and append it to a list, then we'll write it to a data frame

# Eventually swap this out for the list of things that you'd actually like to see in the dataset (maybe the top 200 games?)
gamelist = ["Yukon Salon"]
#gamelist = ['Dwellings of Eldervale']
gamelist = ["Yukon Salon",'Dwellings of Eldervale','On Mars','Gloomhaven']
#gamelist = ["Yukon Salon",'Dwellings of Eldervale']
#gamelist = ['Gloomhaven','Yukon Salon']
#gamelist = ['Gloomhaven']

gamelist = gameList
cnt = 0

for gameid, status in progressList.items():
    try:
        while(not(status)): #If we haven't gotten the results here, get them
            try:
                g = bgg.game(game_id=gameid,rating_comments=True,comments=False)

                try:
                    for game_comment in g.comments:
                        #usrinfo = bgg.user(g.comments[0].commenter) # How to get a single user

                        ratingList.append([g.name, g.id, game_comment.commenter, int(10*float(game_comment.rating))])

                    print('Finished getting the {} ratings for {} successfully'.format(len(g.comments), g.name)) 
                    progressList[gameid] = True
                    status = True
                    break

                except:
                    print("\t Error retrieving rating info for user: {}, skipping!".format(game_comment.commenter))
            except:
                print("Error retrieving game info for {}, retrying!".format(bgg.game(game_id=gameid).name))
                
        else:
            print('Ratings for {} already pulled successfully, continuing'.format(bgg.game(game_id=gameid).name))    
            pass
    except:
        print("Error retrieving game info for {}, skipping!".format(bgg.game(game_id=gameid).name))
        progressList[gameid] = False

Finished getting the 18 ratings for Stardew Valley: The Board Game successfully
Finished getting the 35 ratings for Tiny Epic Dungeons successfully
Finished getting the 15 ratings for So, You've Been Eaten. successfully
Finished getting the 577 ratings for Sleeping Gods successfully
Error retrieving game info for Everdell, retrying!
Error retrieving game info for Everdell, retrying!
Error retrieving game info for Everdell, retrying!
Error retrieving game info for Everdell, retrying!
Error retrieving game info for Everdell, retrying!
Error retrieving game info for Everdell, retrying!
Error retrieving game info for Everdell, retrying!
Error retrieving game info for Everdell, retrying!
Error retrieving game info for Everdell, retrying!
Error retrieving game info for Everdell, retrying!
Error retrieving game info for Everdell, retrying!
Error retrieving game info for Everdell, retrying!
Error retrieving game info for Everdell, retrying!
Error retrieving game info for Everdell, retrying!
Er

In [7]:
# Then we'll write it to a data frame and show some of it
bg_df = pd.DataFrame(ratingList,columns=['Game Name','Game id','User Name', 'Rating'])
print(bg_df)

outfile = '/Users/taylorsmith/Python/BGG/Ratings.csv'
bg_df.to_csv(path_or_buf=outfile, sep='|', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None)

                             Game Name  Game id            User Name  Rating
0       Stardew Valley: The Board Game   332290                Hekuu     100
1       Stardew Valley: The Board Game   332290              Ag0n1st     100
2       Stardew Valley: The Board Game   332290             Rubenmtz     100
3       Stardew Valley: The Board Game   332290             player26     100
4       Stardew Valley: The Board Game   332290         drouinsteven     100
5       Stardew Valley: The Board Game   332290            Monkeydlu     100
6       Stardew Valley: The Board Game   332290          neuxocolata     100
7       Stardew Valley: The Board Game   332290               Hamjul     100
8       Stardew Valley: The Board Game   332290             fstock96     100
9       Stardew Valley: The Board Game   332290              zoldac7     100
10      Stardew Valley: The Board Game   332290              ekiechi     100
11      Stardew Valley: The Board Game   332290            Krisouver      91

In [ ]:
# Need to make a similar database of all the UserNames and associate them with a userID (or at least a uniqueID for the collaborative filtering)